In [25]:
import pandas as pd
import os
from pyproj import Transformer
import pandas as pd
import folium
from folium.plugins import MarkerCluster, HeatMap
import numpy as np

# 1. Limpiar datos erróneos en dataset de Autobuses EMT

## Convertir Separador CSV - Autobuses EMT

Este notebook convierte el archivo `stopsemt.csv` de separador coma (`,`) a punto y coma (`;`) para evitar problemas con campos que contienen comas dentro de comillas.

**Input**: `data/AUTOBUSES/stopsemt.csv` (separador: `,`)

**Output**: `data/AUTOBUSES/stopsemt_clean.csv` (separador: `;`)

In [3]:
df = pd.read_csv("data/AUTOBUSES/stopsemt.csv", sep=",", quotechar='"')
df.to_csv("data/AUTOBUSES/stopsemt_clean.csv", sep=";", index=False, quotechar='"')

## Conversión de Coordenadas UTM a Lat/Long
- **Sistema origen**: UTM Zone 30N (EPSG:25830) - coordenadas proyectadas en metros
- **Sistema destino**: WGS84 (EPSG:4326) - coordenadas geográficas en grados decimales
- **Transformación**: Usa `pyproj.Transformer` para conversión precisa
- **Campos afectados**: `posX` (Este → Longitud), `posY` (Norte → Latitud)

In [12]:
autobuses = pd.read_csv('data/AUTOBUSES/stopsemt_clean.csv', sep=';')

transformer = Transformer.from_crs("EPSG:25830", "EPSG:4326", always_xy=True)

lon, lat = transformer.transform(
    autobuses['posX'].values, 
    autobuses['posY'].values
)

autobuses['posX'] = lon
autobuses['posY'] = lat

autobuses.to_csv('data/AUTOBUSES/stopsemt_clean_coordinates_converted.csv', sep=';', index=False)

# Cargar los datos limpios (salida de hop)

In [23]:
df_metro = pd.read_csv('resultados/metro_procesado.csv', sep=';')
df_autobuses = pd.read_csv('resultados/autobuses_procesado.csv', sep=';')
df_bicimad = pd.read_csv('resultados/bicimad_procesado.csv', sep=';')
df_parkings = pd.read_csv('resultados/parkings_procesado.csv', sep=';')

df_all = pd.concat([df_metro, df_autobuses, df_bicimad])

In [29]:
df_autobuses.head()

,stop_id,stop_name,bus_line,stop_lon,stop_lat,transport_mode
0,4514,Cristo Rey ...,1,-3.716655,40.440258,bus
1,4022,Junta Municipal Moncloa ...,1,-3.717145,40.437624,bus
2,3687,Moncloa ...,1,-3.716809,40.435924,bus
3,737,Altamirano ...,1,-3.716282,40.433990,bus
4,735,Argüelles ...,1,-3.714798,40.431936,bus


In [24]:
print(f"Metro: {len(df_metro)} paradas")
print(f"Autobuses: {len(df_autobuses)} paradas")
print(f"BiciMAD: {len(df_bicimad)} estaciones")
print(f"Parkings: {len(df_parkings)} parkings")
print(f"Total: {len(df_all)} puntos de acceso al transporte")

Metro: 1050 paradas
Autobuses: 12430 paradas
BiciMAD: 626 estaciones
Parkings: 82 parkings
Total: 14106 puntos de acceso al transporte


## Análisis de cobertura

In [30]:
OUTPUT_MAP = 'resultados/mapa_cobertura.html'

def calcular_densidad(df_all):
    print("\nCalculando densidad de estaciones...")
    # Grid de aprox 500m (0.0045 grados lat/lon aprox en Madrid)
    STEP = 0.0045
    
    df_all['lat_bin'] = (df_all['stop_lat'] / STEP).round().astype(int)
    df_all['lon_bin'] = (df_all['stop_lon'] / STEP).round().astype(int)
    
    density = df_all.groupby(['lat_bin', 'lon_bin']).size().reset_index(name='count')
    density['lat_center'] = density['lat_bin'] * STEP
    density['lon_center'] = density['lon_bin'] * STEP
    
    # Top densidades
    print("Top 10 zonas con mayor densidad de transporte:")
    print(density.sort_values('count', ascending=False).head(10))
    return density

def crear_mapa(metro, bus, bici):
    print("\nGenerando mapa...")
    m = folium.Map(location=[40.416775, -3.703790], zoom_start=12, tiles='CartoDB positron')
    
    # Capas
    layer_metro = folium.FeatureGroup(name='Metro (Azul)')
    layer_bus = folium.FeatureGroup(name='Autobús (Verde)')
    layer_bici = folium.FeatureGroup(name='BiciMAD (Rojo)')
    
    # Añadir puntos Metro
    for idx, row in metro.iterrows():
        folium.CircleMarker(
            location=[row['stop_lat'], row['stop_lon']],
            radius=4,
            color='blue',
            fill=True,
            fill_opacity=0.7,
            popup=f"Metro: {row['stop_name']}"
        ).add_to(layer_metro)
        
    # Añadir puntos Bus (Cluster para rendimiento, son muchos)
    bus_cluster = MarkerCluster(name='Autobús (Cluster)').add_to(m)
    # Si se prefiere sin cluster, usar FeatureGroup pero será lento
    # Usamos FeatureGroup para visualización directa si son < 2000, pero son 12k.
    # El usuario pidió "Mapear todos", así que usaremos circles pero con cuidado.
    # Para cumplir "verde", usaremos CircleMarker en el cluster o feature group.
    # Dado el volumen, mejor submuestrear o cluster.
    # Pero el usuario pidió colores específicos. Haremos FeatureGroup pero aviso que puede pesar.
    
    for idx, row in bus.iterrows():
        folium.CircleMarker(
            location=[row['stop_lat'], row['stop_lon']],
            radius=2,
            color='green',
            fill=True,
            fill_opacity=0.5,
            popup=f"Bus: {row['stop_name']}"
        ).add_to(layer_bus)
        
    # Añadir puntos Bici
    for idx, row in bici.iterrows():
        folium.CircleMarker(
            location=[row['stop_lat'], row['stop_lon']],
            radius=3,
            color='red',
            fill=True,
            fill_opacity=0.6,
            popup=f"Bici: {row['stop_name']}"
        ).add_to(layer_bici)

    layer_metro.add_to(m)
    layer_bus.add_to(m)
    layer_bici.add_to(m)
    
    folium.LayerControl().add_to(m)
    
    m.save(OUTPUT_MAP)
    print(f"Mapa guardado en: {OUTPUT_MAP}")


df_all = pd.concat([df_metro, df_autobuses, df_bicimad])

calcular_densidad(df_all)
crear_mapa(df_metro, df_autobuses, df_bicimad)
print("Análisis completado.")



Calculando densidad de estaciones...
Top 10 zonas con mayor densidad de transporte:
     lat_bin  lon_bin  count  lat_center  lon_center
473     8980     -820    122     40.4100     -3.6900
643     8986     -826     91     40.4370     -3.7170
547     8983     -824     85     40.4235     -3.7080
551     8983     -820     78     40.4235     -3.6900
313     8976     -821     74     40.3920     -3.6945
525     8982     -821     68     40.4190     -3.6945
549     8983     -822     66     40.4235     -3.6990
546     8983     -825     64     40.4235     -3.7125
526     8982     -820     63     40.4190     -3.6900
831     8991     -817     60     40.4595     -3.6765

Generando mapa...


KeyError: 'stop_name'